CS244 Assignment 3 - Group 8
===============

* Kevin Rothi (leader)
* J.Y. Ku
* John Lanier 

Github Repository 
https://github.com/JBLanier/cs244fall2017/ 

Both a PDF and an HTML version of this report was submitted, the reason being that the HTML version of this jupyter notebook looks much prettier.

Signal Processing
----------------

Our team used Python for this project. The SciPy library has some great components for signal processing in the scipy.signal library. Applying these components to the signal provided in the correct way proved to be both a challenge and a learning experience.

### Filter Challenges

1. Selecting the correct filter type.
2. Selecting the correct cutoff frequencies.
3. Selecting the correct number of poles to use.
4. Applying the filter to the signal.

First, the filter type wasn't obvious. The team eventually went with a Butter bandpass filter for the heart rate and respiration rate calculations since these signals needed both the high and low frequencies filtered out, since the signal we wanted to isolate occupies a specific band of frequencies. For the SPO2 calculations, after exhaustive experimentation, the team eventually got the best results with a lowpass filter. We theorize that this is because the DC component is a very low frequency wave, and using a bandpass filter would remove it, but more investigation would be needed to confirm this completely.

Selecting the cutoff frequencies was generally the result of experimentation. Too much filtering would remove the peaks, resulting in sine wave like behaviour. This actually made peak detection easier, which this report will cover in the next section, but the tradeoff was that the smooth sine wave would lack detail pertaining to the heart rate which we wanted to keep (the wave would be too smooth).

The correct response curve (pole number) was also not immediately obvious. We eventually settled with a value of 4 since this seemed to give us the best result.

Applying the filter to the signal also proved to be an interesting exercise in signal processing. Using the signal, lfilter method would induce a frequency dependent phase shift to the signal. In the case of the breathing and respiration rate calculations, this wasn't necessarily a bad thing, but with the SPO2 calculation it was imperative that the signal remained in phase if we wanted to use the same peak and valley locations for both signals. We did experiment with working around the phase shift issue be finding the peaks and valleys for both the filtered signals, but this proved to be problematic and this approach was eventually abandoned in favor of using the signal.filtfilt method to apply the filter, which does not induce a phase shift. We also used [Gustafsson’s method](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.signal.filtfilt.html#r216) when applying the filter with filtfilt since we wanted to minimize the transient induced at the beginning of the signal.

Peak Detection
-------------

After the signals had been properly filtered, the next step was to perform peak detection with additional valley detection necessary for the SPO2 calculations. This proved to be as challenging as the filtering step since Python's primary methods for peak and valley detection didn't perform well in practice.

### Peak Detection Challenges

1. Selecting the correct order for signal.argrelextrema
2. Selecting the correct comparator function for signal.argrelextrema
3. Experimenting with peakutils
4. Eventually settling for a new approach (moving subsection of the signal)

SciPy's signal.argrelextrema and it's siblings argrelmax and argrelmin take an argument called "order" which controls how the function looks for peaks, comparing the values around a suspected point using the comparator function out to the value specified with order. The problem is that values of order that are too small would result in false positives, but values too large would miss peaks and valleys. Selecting the correct order proved to be very challenging since accurate peak detection 100% of the time was critical for success, especially in the case of the SPO2 calculations.

On the topic of the comparator function, the choices are the typical greater or less, but also greater_equal and less_equal. If the signal was dramatically filtered, the latter proved to be more effective at correctly finding peaks.

The team also experimented with the peakutils library, but this proved to exhibit the same challenges that were associated with the use of argrelextrema, specifically selecting parameters that were robust to noise and variations in the signal's periodicity.

The team eventually came up with a new method specifically for calculating the SPO2 levels. The entire signal was parcellated into sections by second. This allowed for a much more controlled peak/valley finding technique which would...

1. Do its best to find 2 valleys within a reasonable distance from each other.
2. Do its best to assure that the peak found was between the two valleys.

The code should largely speak for itself, but argrelextrema is called to find the valleys in the signal chunk. Then the peaks are found, and only the peak that landed between the two valleys is kept.

*This works well because it avoids the continuity problems associated with missed valleys and peaks when processing the signal as a whole.*

After we've found the appropriate peaks and valleys, the rest was fairly trivial. For the heart rate and respiration rate calculations, the formula to convert the time between peaks to a per minute rate was simple algebra. The SPO2 calculation was more involved.

### Calculating SPO2

First, we need to find the DC values for the both the IR and red signals. This is accomplished by finding two adjacent valleys and determining their actual values at those points. We then generate a line connecting the two points with numpy's interp function. We then find the peak between the two valleys, and ascertain the interpolated value where the peak is. This is the DC value. The AC value is simply the difference between the value at the peak and the DC value.

This was easily the most challenging part of the assignment. Applying the appropriate filtering and finding the peaks/valleys represented the bulk of the work done, but the final result seems satisfactory. 

### Final Notes

Something you might notice is that in our results we have the heart rate and respiration rate as peak to peak values. For SPO2, we have it broken down by second. This is a consequence of the two different methods we used to derive our results. The Heart Rate, Respiration Rate, and SPO2 values in each row of the csv do not correspond to the explicit time values on the same rows, nor do they correspond to each other in time.


In [2]:
from numpy import genfromtxt
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import math
%matplotlib inline

per_data = genfromtxt('data.csv', delimiter=',') #pull in the data from the csv (generates matrix)

def column(matrix, i): #def a function to pull out the columns from the matrix
    return [row[i] for row in matrix]
    
def getRatio(acRed, dcIR, acIR, dcRed):
    return float(acRed * dcIR) / (acIR * dcRed)

def calculateSPO2(R):
    return (-45.060 * math.pow(R, 2)) + (30.354 * R) + 94.845

def generateWindow(second):
    return second * 50, (second + 3) * 50

time = column(per_data, 0)[1:] #use the function we came up with earlier to pull out the columns for time, ir, red
ir = column(per_data, 2)[1:]
red = column(per_data, 1)[1:]

b, a = signal.butter(4, [0.0340, 0.0354], btype='bandpass') #filter for heart rate
hrdata = signal.lfilter(b, a, ir)
b, a = signal.butter(4, 0.8, btype='lowpass') #filter for SPO2, just removing hf noise
filteredReddata = signal.filtfilt(b, a, red, method='gust') #using filtfilt to avoid phase shift
filteredIRdata = signal.filtfilt(b, a, ir, method='gust')
b, a = signal.butter(4, [0.00972, 0.0113], btype='bandpass') #filter for breathing rate
rrdata = signal.lfilter(b, a, ir)

hrPeaks = signal.argrelextrema(hrdata, np.greater_equal, order=26)[0] #find the peaks for HR and breathing rate
rrPeaks = signal.argrelextrema(rrdata, np.greater_equal, order=100)[0]

print('---------- PRINTING INSTANT HR VALUES -----------')
for peakIndex in range(len(hrPeaks) - 1):
    distance = hrPeaks[peakIndex + 1] - hrPeaks[peakIndex]
    print(60 / (distance * 0.02))
print('---------- END HR VALUES ---------------')
    
print('---------- PRINTING INSTANT RR VALUES -----------')
for peakIndex in range(len(rrPeaks) - 1):
    distance = rrPeaks[peakIndex + 1] - rrPeaks[peakIndex]
    print(60 / (distance * 0.02))
print('---------- END RR VALUES ---------------')

redDC = []
redAC = []
irDC = []
irAC = []

for second in range(89):
    lb, ub = generateWindow(second) #isolate the chunk we're analyzing now, making sure chunk is big enough to get 2 valleys
    redSlice = filteredReddata[lb:ub]
    irSlice = filteredIRdata[lb:ub] #filtered data
    valleys = signal.argrelmin(redSlice, order=10)[0] #find peaks and valleys
    peaks = signal.argrelmax(redSlice, order=10)[0]
    valley1 = valleys[0] + lb
    valley2 = valleys[1] + lb
    peak = peaks[0] + lb
    if (peak < valley1): #to make sure the peak is between the valleys
        peak = peaks[1] + lb
    dcir = np.interp(peak, [valley1, valley2], [filteredIRdata[valley1], filteredIRdata[valley2]])
    dcred = np.interp(peak, [valley1, valley2], [filteredReddata[valley1], filteredReddata[valley2]])
    irDC += [dcir]
    irAC += [filteredIRdata[peak] - dcir]
    redDC += [dcred]
    redAC += [filteredReddata[peak] - dcred]
    
print('---------- PRINTING SPO2 VALUES -----------')
for iteration in range(len(redDC)):
    print(calculateSPO2(getRatio(redAC[iteration], irDC[iteration], irAC[iteration], redDC[iteration])))
print('---------- END SPO2 VALUES -----------')

---------- PRINTING INSTANT HR VALUES -----------
57.6923076923
54.5454545455
52.6315789474
52.6315789474
52.6315789474
51.724137931
52.6315789474
51.724137931
52.6315789474
51.724137931
52.6315789474
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
51.724137931
51.724137931
52.6315789474
51.724137931
51.724137931
51.724137931
51.724137931
50.8474576271
50.0
44.776119403
46.1538461538
50.0
50.8474576271
51.724137931
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
52.6315789474
51.724137931
51.724137931
52.6315789474
51.724137931
52.6315789474
52.6315789474
51.724137931
52.6315789474
52.6315789474
53.5714285714
55.5555555556
60.0
58.8235294118
54.5454545455
53.5714285714
52.6